In [34]:
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cluster import KMeans
import numpy as np

In [29]:
customer_df = pd.read_csv('/Users/a1111/Desktop/group_project/CRR/db/Customer.csv')
order_df = pd.read_csv('/Users/a1111/Desktop/group_project/CRR/db/Order.csv')
product_df = pd.read_csv('/Users/a1111/Desktop/group_project/CRR/db/Product.csv')

In [30]:
# Preliminary data inspection to decide on the temporal split
order_df['OrderDate'] = pd.to_datetime(order_df['OrderDate'])
latest_order_date = order_df['OrderDate'].max()
earliest_order_date = order_df['OrderDate'].min()

(earliest_order_date, latest_order_date)

(Timestamp('2022-04-10 00:00:00'), Timestamp('2024-03-31 00:00:00'))

In [31]:
# Setting the split date
split_date = pd.to_datetime('2023-12-31')

# Merging Order and Product on ProductID
order_product_df = pd.merge(order_df, product_df, on='ProductID')

# Calculating total order value for each order
order_product_df['OrderValue'] = order_product_df['Quantity'] * order_product_df['Price']

# Merging with Customer dataset on CustomerID 
full_df = pd.merge(order_product_df, customer_df, on='CustomerID')

# Feature Engineering
# Feature 1: Total Order Count by Customer (up to split_date for training set)
total_order_count = full_df[full_df['OrderDate'] <= split_date].groupby('CustomerID')['OrderID'].nunique().reset_index(name='TotalOrderCount')

# Feature 2: Average Order Value by Customer (up to split_date for training set)
average_order_value = full_df[full_df['OrderDate'] <= split_date].groupby('CustomerID')['OrderValue'].mean().reset_index(name='AverageOrderValue')

# Feature 3: Days Since Last Order (from split_date, as this feature is crucial for defining churn)
# For the training set, we'll calculate days since the last order up to the split date
full_df['DaysSinceLastOrder'] = (split_date - pd.to_datetime(full_df['OrderDate'])).dt.days
days_since_last_order = full_df[full_df['OrderDate'] <= split_date].groupby('CustomerID')['DaysSinceLastOrder'].min().reset_index()

# Feature 4: Product Diversity (Number of Unique Products Purchased)
product_diversity = full_df[full_df['OrderDate'] <= split_date].groupby('CustomerID')['ProductID'].nunique().reset_index(name='ProductDiversity')

# Merging all features into a single DataFrame for the training set
features_df_train = customer_df[['CustomerID']].merge(total_order_count, on='CustomerID', how='left')\
                                                .merge(average_order_value, on='CustomerID', how='left')\
                                                .merge(days_since_last_order, on='CustomerID', how='left')\
                                                .merge(product_diversity, on='CustomerID', how='left')

# Fill NaN values with 0 for customers who didn't place any order
features_df_train.fillna(0, inplace=True)

# Adjusting the churn definition for the training set
# Let's consider customers as churned if they haven't made any order in the last 90 days up to the split date
features_df_train['Churn'] = features_df_train['DaysSinceLastOrder'] > 90

features_df_train.head(), features_df_train['Churn'].value_counts()


(   CustomerID  TotalOrderCount  AverageOrderValue  DaysSinceLastOrder  \
 0           1              1.0        1544.520000               402.0   
 1           2              2.0        1048.835000               161.0   
 2           3              1.0        3538.560000               421.0   
 3           4              1.0          85.640000               143.0   
 4           5              3.0         969.436667               488.0   
 
    ProductDiversity  Churn  
 0               1.0   True  
 1               2.0   True  
 2               1.0   True  
 3               1.0   True  
 4               3.0   True  ,
 True     64
 False    36
 Name: Churn, dtype: int64)

In [32]:
# Preparing the features and target variable for model training
X_train = features_df_train[['TotalOrderCount', 'AverageOrderValue', 'DaysSinceLastOrder', 'ProductDiversity']]
y_train = features_df_train['Churn']

# Standardizing the features for the training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Logistic Regression model with L2 regularization
model_lr = LogisticRegression(penalty='l2', C=1.0, random_state=42)

# Cross-validation to evaluate model performance
cv_scores = cross_val_score(model_lr, X_train_scaled, y_train, cv=5, scoring='accuracy')

cv_scores.mean(), cv_scores.std()

(0.99, 0.020000000000000018)

In [33]:
# Initializing the Gradient Boosting Classifier
model_gb = GradientBoostingClassifier(random_state=42)

# Cross-validation to evaluate model performance
cv_scores_gb = cross_val_score(model_gb, X_train_scaled, y_train, cv=5, scoring='accuracy')

cv_scores_gb.mean(), cv_scores_gb.std()

(1.0, 0.0)

In [35]:
# Number of clusters
k = 5

# Initializing K-Means with the chosen number of clusters and fitting it to the data
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(X_train_scaled)

# Predicting the cluster labels for the dataset
cluster_labels = kmeans.predict(X_train_scaled)

# Adding the cluster labels to our features DataFrame
features_df_train['Cluster'] = cluster_labels

# Analyzing the distribution of customers across clusters
cluster_distribution = np.bincount(cluster_labels)

cluster_distribution, features_df_train.groupby('Cluster').mean()

/Users/a1111/opt/anaconda3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


(array([25, 11, 16, 12, 36]),
          CustomerID  TotalOrderCount  AverageOrderValue  DaysSinceLastOrder  \
 Cluster                                                                       
 0         46.880000         1.960000        1100.070467          341.360000   
 1         48.909091         5.636364        1588.069883           79.636364   
 2         59.312500         1.687500        2917.006042          291.062500   
 3         58.833333         0.333333         189.311667           21.083333   
 4         46.805556         3.416667        1352.027958           91.305556   
 
          ProductDiversity     Churn  
 Cluster                              
 0                1.920000  1.000000  
 1                5.363636  0.363636  
 2                1.625000  0.937500  
 3                0.333333  0.083333  
 4                3.250000  0.527778  )